In [138]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from imblearn.pipeline import Pipeline as ImbPipeline
from imblearn.over_sampling import SMOTE
import pickle

# DataFrame

In [139]:
import pandas as pd
import numpy as np

class DataFrameProcessor:
    def __init__(self, dataframe):
        """
        Inicializa a classe com um dataframe.
        
        Args:
            dataframe (pd.DataFrame): O dataframe a ser processado.
        """
        self.dataframe = dataframe

    def order(self):
        """
        Reordena as colunas do dataframe de acordo com as categorias definidas.
        
        Returns:
            pd.DataFrame: Dataframe com colunas reordenadas.
        """
        self.dataframe = self.dataframe[[
            # 1. Variáveis Demográficas:
            'id', 'idade', 'sexo', 'dependentes', 'escolaridade', 'estado_civil', 'grupo_etario', 'idade_faixa',
            'idade_estado_civil', 'escolaridade_renda',

            # 2. Variáveis Financeiras:
            'salario_anual', 'limite_credito', 'renda_alta',

            # 3. Variáveis de Tempo:
            'meses_de_relacionamento', 'interacoes_12m', 'meses_inativo_12m',  'valor_transacoes_12m',
            'qtd_transacoes_12m', 'transacoes_por_mes', 'meses_ativos_12m',

            # 4. Variáveis do cartão:
            'tipo_cartao', 'taxa_utilizacao_credito', 'valor_medio_transacao', 'frequencia_interacao', 'qtd_produtos',

            # 5. Variável de default:
            'default'
        ]]
        return self.dataframe

    def columns_type(self):
        """
        Converte as colunas categóricas para o tipo 'category'.
        
        Returns:
            pd.DataFrame: Dataframe com colunas categóricas convertidas.
        """
        category_list = list(self.dataframe.select_dtypes('object').columns)
        for col in category_list:
            self.dataframe[col] = self.dataframe[col].astype('category')
        return self.dataframe

    def attribute_engineering(self):
        """
        Realiza a engenharia de atributos no dataframe.
        
        Returns:
            pd.DataFrame: Dataframe após a engenharia de atributos.
        """
        fn = lambda valor: float(valor.replace(".", "").replace(",", "."))
        self.dataframe['valor_transacoes_12m'] = self.dataframe['valor_transacoes_12m'].apply(fn)
        self.dataframe['limite_credito'] = self.dataframe['limite_credito'].apply(fn)

        # Definindo as faixas etárias
        bins = [26, 31, 37, 45, 51, 59, 65, float('inf')]
        labels = ['26-31', '32-37', '38-45', '46-51', '52-59', '60-65', '66+']
        self.dataframe['idade_faixa'] = pd.cut(self.dataframe['idade'], bins=bins, labels=labels, right=True)

        self.dataframe['taxa_utilizacao_credito'] = self.dataframe['valor_transacoes_12m'] / self.dataframe['limite_credito']
        self.dataframe['valor_medio_transacao'] = self.dataframe['valor_transacoes_12m'] / self.dataframe['qtd_transacoes_12m']
        self.dataframe['transacoes_por_mes'] = self.dataframe['qtd_transacoes_12m'] / 12

        self.dataframe['meses_ativos_12m'] = 12 - self.dataframe['meses_inativo_12m']
        self.dataframe['interacoes_12m'] = self.dataframe['iteracoes_12m']
        self.dataframe.drop(columns='iteracoes_12m', inplace=True)
        self.dataframe['frequencia_interacao'] = self.dataframe['interacoes_12m'] / self.dataframe['meses_ativos_12m']

        self.dataframe['idade_estado_civil'] = self.dataframe['idade'].astype(str) + ' - ' + self.dataframe['estado_civil']
        self.dataframe['escolaridade_renda'] = self.dataframe['escolaridade'] + ' - ' + self.dataframe['salario_anual']

        # Definindo as faixas etárias ajustadas
        bins = [26, 30, 60, float('inf')]
        labels = ['Jovem', 'Adulto', 'Idoso']
        self.dataframe['grupo_etario'] = pd.cut(self.dataframe['idade'], bins=bins, labels=labels, right=True)

        self.dataframe['renda_alta'] = np.where(self.dataframe['salario_anual'].isin(['$80K - $120K', '$120K +']), 1, 0)

        # Concatenar colunas categóricas em uma nova coluna 'perfil'
        #self.dataframe['perfil'] = (self.dataframe['sexo'].astype(str) + '_' + self.dataframe['escolaridade'].astype(str) + '_' 
                                    #+ self.dataframe['estado_civil'].astype(str) + '_' + self.dataframe['salario_anual'].astype(str))

        # Reordenando colunas e convertendo tipos
        self.dataframe = self.order()
        self.dataframe = self.columns_type()

        return self.dataframe

    def process(self):
        """
        Executa todo o processamento no dataframe.
        
        Returns:
            pd.DataFrame: Dataframe processado.
        """
        self.attribute_engineering()
        return self.dataframe

url = 'https://raw.githubusercontent.com/enzoschitini/Data-Science-Portfolio/main/01%20An%C3%A1lise%20explorat%C3%B3ria%20e%20limpeza%20de%20dados/EBAC/credito.csv'

df = pd.read_csv(url, na_values='na')#.drop(columns='Unnamed: 0')

# Criar uma instância da classe DataFrameProcessor
processor = DataFrameProcessor(df)

# Processar o dataframe
df_processado = processor.process()

# Mostrar as primeiras linhas do dataframe processado
df_processado.head()
#df_processado.to_csv('DF_model2.csv')

,id,idade,sexo,dependentes,escolaridade,estado_civil,grupo_etario,idade_faixa,idade_estado_civil,escolaridade_renda,...,valor_transacoes_12m,qtd_transacoes_12m,transacoes_por_mes,meses_ativos_12m,tipo_cartao,taxa_utilizacao_credito,valor_medio_transacao,frequencia_interacao,qtd_produtos,default
0,768805383,45,M,3,ensino medio,casado,Adulto,38-45,45 - casado,ensino medio - $60K - $80K,...,1144.90,42,3.500000,11,blue,0.090210,27.259524,0.272727,5,0
1,818770008,49,F,5,mestrado,solteiro,Adulto,46-51,49 - solteiro,mestrado - menos que $40K,...,1291.45,33,2.750000,11,blue,0.156407,39.134848,0.181818,6,0
2,713982108,51,M,3,mestrado,casado,Adulto,46-51,51 - casado,mestrado - $80K - $120K,...,1887.72,20,1.666667,11,blue,0.552197,94.386000,0.000000,4,0
3,769911858,40,F,4,ensino medio,NaN,Adulto,38-45,NaN,ensino medio - menos que $40K,...,1171.56,20,1.666667,8,blue,0.353622,58.578000,0.125000,3,0
4,709106358,40,M,3,sem educacao formal,casado,Adulto,38-45,40 - casado,sem educacao formal - $60K - $80K,...,816.08,28,2.333333,11,blue,0.173037,29.145714,0.000000,5,0


# Model

In [140]:
df = pd.read_csv('../DF_model.csv').drop(columns=['Unnamed: 0'])
df.head()

,id,idade,sexo,dependentes,escolaridade,estado_civil,grupo_etario,idade_faixa,idade_estado_civil,escolaridade_renda,...,qtd_transacoes_12m,transacoes_por_mes,meses_ativos_12m,tipo_cartao,taxa_utilizacao_credito,valor_medio_transacao,frequencia_interacao,qtd_produtos,default,cat
0,768805383,45,M,3,ensino medio,casado,Adulto,38-45,45 - casado,ensino medio - $60K - $80K,...,42,3.500000,11,blue,0.090210,27.259524,0.272727,5,0,M_ensino medio_casado_$60K - $80K
1,818770008,49,F,5,mestrado,solteiro,Adulto,46-51,49 - solteiro,mestrado - menos que $40K,...,33,2.750000,11,blue,0.156407,39.134848,0.181818,6,0,F_mestrado_solteiro_menos que $40K
2,713982108,51,M,3,mestrado,casado,Adulto,46-51,51 - casado,mestrado - $80K - $120K,...,20,1.666667,11,blue,0.552197,94.386000,0.000000,4,0,M_mestrado_casado_$80K - $120K
3,709106358,40,M,3,sem educacao formal,casado,Adulto,38-45,40 - casado,sem educacao formal - $60K - $80K,...,28,2.333333,11,blue,0.173037,29.145714,0.000000,5,0,M_sem educacao formal_casado_$60K - $80K
4,713061558,44,M,2,mestrado,casado,Adulto,38-45,44 - casado,mestrado - $40K - $60K,...,24,2.000000,11,blue,0.271292,45.336250,0.181818,3,0,M_mestrado_casado_$40K - $60K


In [141]:
with open('LGBMClassifier.pkl', 'rb') as file:
    loaded_model = pickle.load(file)

display(df.head())

DF_ML = pd.get_dummies(df, drop_first=True)

# Separazione delle caratteristiche e della variabile target
y = DF_ML.default
X = DF_ML.drop(columns='default', axis=1)

# Suddivisione in set di addestramento e test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=100)

# Standardizzazione delle caratteristiche
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)                   
X_test = scaler.transform(X)

y_pred = loaded_model.predict(X_test)
print(y_pred)
print(len(y_pred))

df['predict'] = y_pred
df.head()

,id,idade,sexo,dependentes,escolaridade,estado_civil,grupo_etario,idade_faixa,idade_estado_civil,escolaridade_renda,...,qtd_transacoes_12m,transacoes_por_mes,meses_ativos_12m,tipo_cartao,taxa_utilizacao_credito,valor_medio_transacao,frequencia_interacao,qtd_produtos,default,cat
0,768805383,45,M,3,ensino medio,casado,Adulto,38-45,45 - casado,ensino medio - $60K - $80K,...,42,3.500000,11,blue,0.090210,27.259524,0.272727,5,0,M_ensino medio_casado_$60K - $80K
1,818770008,49,F,5,mestrado,solteiro,Adulto,46-51,49 - solteiro,mestrado - menos que $40K,...,33,2.750000,11,blue,0.156407,39.134848,0.181818,6,0,F_mestrado_solteiro_menos que $40K
2,713982108,51,M,3,mestrado,casado,Adulto,46-51,51 - casado,mestrado - $80K - $120K,...,20,1.666667,11,blue,0.552197,94.386000,0.000000,4,0,M_mestrado_casado_$80K - $120K
3,709106358,40,M,3,sem educacao formal,casado,Adulto,38-45,40 - casado,sem educacao formal - $60K - $80K,...,28,2.333333,11,blue,0.173037,29.145714,0.000000,5,0,M_sem educacao formal_casado_$60K - $80K
4,713061558,44,M,2,mestrado,casado,Adulto,38-45,44 - casado,mestrado - $40K - $60K,...,24,2.000000,11,blue,0.271292,45.336250,0.181818,3,0,M_mestrado_casado_$40K - $60K


[0 0 0 ... 0 1 1]
7081


,id,idade,sexo,dependentes,escolaridade,estado_civil,grupo_etario,idade_faixa,idade_estado_civil,escolaridade_renda,...,transacoes_por_mes,meses_ativos_12m,tipo_cartao,taxa_utilizacao_credito,valor_medio_transacao,frequencia_interacao,qtd_produtos,default,cat,predict
0,768805383,45,M,3,ensino medio,casado,Adulto,38-45,45 - casado,ensino medio - $60K - $80K,...,3.500000,11,blue,0.090210,27.259524,0.272727,5,0,M_ensino medio_casado_$60K - $80K,0
1,818770008,49,F,5,mestrado,solteiro,Adulto,46-51,49 - solteiro,mestrado - menos que $40K,...,2.750000,11,blue,0.156407,39.134848,0.181818,6,0,F_mestrado_solteiro_menos que $40K,0
2,713982108,51,M,3,mestrado,casado,Adulto,46-51,51 - casado,mestrado - $80K - $120K,...,1.666667,11,blue,0.552197,94.386000,0.000000,4,0,M_mestrado_casado_$80K - $120K,0
3,709106358,40,M,3,sem educacao formal,casado,Adulto,38-45,40 - casado,sem educacao formal - $60K - $80K,...,2.333333,11,blue,0.173037,29.145714,0.000000,5,0,M_sem educacao formal_casado_$60K - $80K,0
4,713061558,44,M,2,mestrado,casado,Adulto,38-45,44 - casado,mestrado - $40K - $60K,...,2.000000,11,blue,0.271292,45.336250,0.181818,3,0,M_mestrado_casado_$40K - $60K,0
